<a href="https://colab.research.google.com/github/Aakash326/ML-models/blob/main/Resume_GenAi__proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install pydantic
from pydantic import BaseModel

In [2]:
class User(BaseModel):
    name: str
    age: int
    email: str

In [3]:
user = User(name = "Mira", age = 30, email = "mira@gmail.com")
print(user.json())

{"name":"Mira","age":30,"email":"mira@gmail.com"}


/tmp/ipython-input-3-2340500492.py:2: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(user.json())


In [4]:
user = User(name = "Mira", age = "not-a-number", email = "mira@gmail.com")
print(user.json())

ValidationError: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not-a-number', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

In [2]:
import os
import google.generativeai as genai
from openai import OpenAI  # Make sure you have the latest openai package (pip install --upgrade openai)
# from dotenv import load_dotenv
import json
from typing import List, Dict, Union, Optional, Any
from IPython.display import display, Markdown

In [3]:
from google.colab import userdata
from openai import OpenAI

# ✅ Load the secret from Colab
openai_api_key = userdata.get("OPENAI_API_KEY")  # must match exactly what you named it in the Secrets panel

# ✅ Initialize the OpenAI client
client = OpenAI(api_key=openai_api_key)

In [78]:
gemini_ai = userdata.get("GEMINI_API_KEY")
# Initialize the model
genai.configure(api_key=gemini_ai)
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

In [4]:
class scientist(BaseModel):
  name: str
  field: str
  known_for: list[str]
  birth_year: int

In [24]:
prompt = """
Give me a JSON object with details about a famous scientist.
Include the following fields: name, field, known_for, and birth_year.
"""

# Let's make the API call to OpenAI
# Note that we used "openai_client.beta.chat.completions.parse( )" since we want to parse it into a structured output instead of just plain text
response = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0,
    response_format=scientist,  # Must be used with `.parse()`, not `.create()`
    max_tokens=300
)

In [25]:
response.choices[0].message.content

'{"name":"Albert Einstein","field":"Physics","known_for":["Theory of Relativity","Photoelectric Effect","Mass-energy equivalence (E=mc²)"],"birth_year":1879}'

In [23]:
json.loads(response.choices[0].message.content)

{'name': 'Albert Einstein',
 'field': 'Physics',
 'known_for': ['Theory of Relativity',
  'Photoelectric Effect',
  'Mass-energy equivalence (E=mc²)'],
 'birth_year': 1879}

In [5]:
def print_markdown(text):
    """Displays text as Markdown."""
    display(Markdown(text))

In [6]:
resume_text = """
**Jessica Brown**
jessica.brown@email.com | (416) 555-7890 | linkedin.com/in/jessicabrown

**Summary**
Marketing professsional with 2 years of experience assisting in digital campaigns, content creation, and social media activities. Comfortable handling multiple tasks and providing general marketing support.

**Experience**

**Marketing Asssistant | Brewster Coffee Co. | Toronto, ON | Jan 2022 – Present**
- Assisted with digital marketing campaigns including email and social media.
- Created blog posts and social media updates to improve audience engagement.
- Managed social media accounts and grew follower numbers.
- Supported coordination of marketing events.
- Conducted market research and competitor analysis.

**Skils**
- Digital Marketing (SEO basics, Email Marketing)
- Social Media Tools (Hootsuite, Buffer)
- Microsoft Office Suite, Google Workspace
- Basic knowledge of Adobe Photoshop

**Education**
**Bachelor of Commerce, Marketing** | Ryerson University (now Toronto Metropolitan University), Toronto, ON | May 2021
"""

In [13]:
job_description_text = """
# Job Title: Digital Marketing Specialist

**Company:** BrightWave Digital Agency

**Location:** Toronto, ON

## About Us:
BrightWave Digital Agency creates digital marketing campaigns for a variety of clients. We are looking for a Digital Marketing Specialist to join our team and assist in managing campaigns.

## Responsibilities:
- Assist in planning and executing digital marketing campaigns (SEO, SEM, social media, email).
- Use Google Analytics to measure performance and prepare basic performance reports.
- Support social media management tasks including content scheduling and community engagement.
- Perform keyword research and assist in optimizing content for SEO.
- Work with designers to help coordinate campaign materials.
- Keep informed about current digital marketing trends.

## Qualifications:
- Bachelor's degree in Marketing, Communications, or similar.
- 2+ years of digital marketing experience.
- Familiarity with SEO, SEM, Google Analytics, and social media.
- Ability to interpret basic marketing data.
- Good communication and writing skills.
- Knowledge of CRM systems (e.g., HubSpot) helpful.
- Experience with Adobe Creative Suite is beneficial.
"""

In [7]:
print_markdown("**--- Original Resume ---**")
print_markdown(resume_text)

**--- Original Resume ---**


**Jessica Brown**  
jessica.brown@email.com | (416) 555-7890 | linkedin.com/in/jessicabrown

**Summary**  
Marketing professsional with 2 years of experience assisting in digital campaigns, content creation, and social media activities. Comfortable handling multiple tasks and providing general marketing support.

**Experience**

**Marketing Asssistant | Brewster Coffee Co. | Toronto, ON | Jan 2022 – Present**
- Assisted with digital marketing campaigns including email and social media.
- Created blog posts and social media updates to improve audience engagement.
- Managed social media accounts and grew follower numbers.
- Supported coordination of marketing events.
- Conducted market research and competitor analysis.

**Skils**
- Digital Marketing (SEO basics, Email Marketing)
- Social Media Tools (Hootsuite, Buffer)
- Microsoft Office Suite, Google Workspace
- Basic knowledge of Adobe Photoshop

**Education**  
**Bachelor of Commerce, Marketing** | Ryerson University (now Toronto Metropolitan University), Toronto, ON | May 2021


In [8]:
def openai(prompt:str):
          #  response_format:Optional[dict] =None)->str | dict:
  try:
    if response_format is None:
      response=client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
              {"role":"system","content":"you are a helpful assistant specializing in resume writing and career advice."},
              {"role":"user","content":prompt}
          ],
      temperature=0.7,
      max_tokens=1500
      )

      return response.choices[0].message.content

    else:
            completion =client.beta.chat.completions.parse(
                model = model,  # Make sure to use a model that supports parse
                messages = [
                    # Same system and user messages as above
                    {
                        "role": "system",
                        "content": "You are a helpful assistant specializing in resume writing and career advice.",
                    },
                    {"role": "user", "content": prompt},
                ],
                temperature = 0.7,
                response_format = response_format)

            # Return the parsed structured output
            return completion.choices[0].message.parsed

  except Exception as e:
        # Error handling to prevent crashes
        return f"Error generating text: {e}"

In [9]:
def openai(prompt: str, response_format=None, model="gpt-4o-mini", temperature=0.7):
    try:
        if response_format is None:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant specializing in resume writing."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=1500
            )
            return response.choices[0].message.content
        else:
            completion = client.beta.chat.completions.parse(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant specializing in resume writing."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                response_format=response_format,  # Fixed parameter name
                max_tokens=1500
            )
            return completion.choices[0].message.parsed
    except Exception as e:
        return f"Error generating text: {e}"

In [10]:
prompt = f"""
Context:
You are a professional resume writer helping a candidate tailor their resume for a specific job opportunity. The resume and job description are provided below.

Instruction:
Enhance the resume to make it more impactful. Focus on:
- Highlighting relevant skills and achievements.
- Using strong action verbs and quantifiable results where possible.
- Rewriting vague or generic bullet points to be specific and results-driven.
- Emphasizing experience and skills most relevant to the job description.
- Reorganizing sections if necessary to better match the job requirements.

Resume:
{resume_text}

Output:
Provide a revised and improved version of the resume that is well-formatted. Only return the updated resume.
"""

In [59]:
openai_output = openai(prompt)

# Display the results
print_markdown("#### OpenAI Response:")
print_markdown(openai_output)

#### OpenAI Response:

**Jessica Brown**  
jessica.brown@email.com | (416) 555-7890 | linkedin.com/in/jessicabrown

---

**Summary**  
Results-driven Marketing Professional with 2 years of experience in executing digital marketing strategies and enhancing brand visibility through targeted content creation and social media engagement. Proven ability to manage multiple projects simultaneously and deliver measurable results in fast-paced environments.

---

**Experience**

**Marketing Assistant | Brewster Coffee Co. | Toronto, ON | Jan 2022 – Present**  
- Spearheaded digital marketing campaigns, increasing email open rates by 25% and social media engagement by 40% through strategic content development and audience targeting.
- Developed and published over 30 blog posts and social media updates, driving a 15% increase in website traffic and enhancing community interaction.
- Managed and optimized social media accounts, successfully growing follower base from 1,000 to 3,500 in under a year through innovative engagement strategies.
- Coordinated and executed marketing events, resulting in a 20% increase in customer attendance and brand awareness.
- Conducted comprehensive market research and competitor analysis, providing actionable insights that led to the refining of marketing strategies and improved campaign performance.

---

**Skills**  
- Digital Marketing: SEO fundamentals, Email Marketing, Campaign Analytics  
- Social Media Management: Hootsuite, Buffer, Content Scheduling  
- Proficient in Microsoft Office Suite and Google Workspace  
- Basic Graphic Design: Adobe Photoshop  

---

**Education**  
**Bachelor of Commerce, Marketing** | Toronto Metropolitan University, Toronto, ON | May 2021  

---

In [79]:
def gemini_generate(prompt: str, temperature: float = 0.7, max_output_tokens: int = 1500) -> str:
    try:
        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=temperature,
                max_output_tokens=max_output_tokens
            ),
        )
        # ✅ Correctly access generated content using parts
        if not response.candidates or not response.candidates[0].content.parts:
            reason = response.candidates[0].finish_reason if response.candidates else "unknown"
            return f"⚠️ No content generated. finish_reason = {reason}"

        output_text = "".join([p.text for p in response.candidates[0].content.parts])
        return output_text

    except Exception as e:
        return f"❌ Error generating text: {e}"


In [11]:
def analyze_resume_against_job_description(job_description_text: str, resume_text: str, model: str = "openai") -> str:
    """
    Analyze the resume against the job description and return a structured comparison.

    Args:
        job_description_text (str): The job description text.
        resume_text (str): The candidate's resume text.
        model (str): The model to use for analysis ("openai" or "gemini").

    Returns:
        str: A clear, structured comparison of the resume and job description.
    """
    # This prompt instructs the AI to act as a career advisor and analyze how well the resume matches the job description
    # It asks for a structured analysis with 4 specific sections: requirements, matches, gaps, and strengths
    prompt = f"""
    Context:
    You are a career advisor and resume expert. Your task is to analyze a candidate's resume against a specific job description to assess alignment and identify areas for improvement.

    Instruction:
    Review the provided Job Description and Resume. Identify key skills, experiences, and qualifications in the Job Description and compare them to what's present in the Resume. Provide a structured analysis with the following sections:
    1. **Key Requirements from Job Description:** List the main skills, experiences, and qualifications sought by the employer.
    2. **Relevant Experience in Resume:** List the skills and experiences from the resume that match or align closely with the job requirements.
    3. **Gaps/Mismatches:** Identify important skills or qualifications from the Job Description that are missing, unclear, or underrepresented in the Resume.
    4. **Potential Strengths:** Highlight any valuable skills, experiences, or accomplishments in the resume that are not explicitly requested in the job description but could strengthen the application.

    Job Description:

    {job_description_text}

    Resume:

    {resume_text}

    Output:
    Return a clear, structured comparison with the four sections outlined above.
    """

    # This conditional block selects which AI model to use based on the 'model' parameter
    if model == "openai":
      # Uses OpenAI's model to generate the gap analysis with moderate creativity (temperature=0.7)
      gap_analysis = openai(prompt, temperature=0.7)
    elif model == "gemini":
        # Uses Google's Gemini model with less creativity (temperature=0.5) for more focused results
        gap_analysis = gemini_generate(prompt, temperature=0.5)
    else:
        # Raises an error if an invalid model name is provided
        raise ValueError(f"Invalid model: {model}")

    # Returns the generated gap analysis text
    return gap_analysis

In [15]:
gap_analysis_openai = analyze_resume_against_job_description(job_description_text,
                                                             resume_text,
                                                             model = "openai")

In [16]:
print_markdown("#### Open AI:")
print_markdown(gap_analysis_openai)

#### Open AI:

### 1. Key Requirements from Job Description:
- **Education:** Bachelor's degree in Marketing, Communications, or similar.
- **Experience:** 2+ years of digital marketing experience.
- **Skills:**
  - Proficiency in SEO, SEM, social media, and email marketing.
  - Familiarity with Google Analytics and ability to interpret marketing data.
  - Good communication and writing skills.
  - Knowledge of CRM systems (e.g., HubSpot).
  - Experience with Adobe Creative Suite is beneficial.
- **Responsibilities:**
  - Planning and executing digital marketing campaigns.
  - Supporting social media management including content scheduling and community engagement.
  - Performing keyword research and optimizing content for SEO.
  - Coordinating with designers for campaign materials.
  - Staying informed about current digital marketing trends.

### 2. Relevant Experience in Resume:
- **Education:** Bachelor's degree in Marketing (Bachelor of Commerce, Marketing from Ryerson University).
- **Experience:** 2 years as a Marketing Assistant, which includes:
  - Assisting with digital marketing campaigns (email and social media).
  - Creating content for blogs and social media, which contributes to audience engagement.
  - Managing social media accounts and growing follower numbers.
  - Conducting market research, which aligns with a foundational aspect of campaign planning.
- **Skills:**
  - Basic understanding of SEO and experience with email marketing.
  - Experience with social media tools (Hootsuite, Buffer).
  - Basic knowledge of Adobe Photoshop (though not the full Creative Suite).

### 3. Gaps/Mismatches:
- **Experience Requirement:** The resume shows only 2 years of experience, which meets the minimum requirement but does not exceed or demonstrate advanced capabilities in digital marketing.
- **Technical Skills:**
  - Lack of explicit experience with SEM and Google Analytics; the resume does not mention familiarity with these tools, which are crucial for measuring campaign performance.
  - No mention of CRM systems like HubSpot, which is noted as helpful in the job description.
  - The resume indicates only basic SEO knowledge, whereas the job may require more advanced skills in keyword research and content optimization.
- **Communication Skills:** While the resume states good communication and writing skills, it would be beneficial to provide specific examples or achievements that demonstrate these abilities.
  
### 4. Potential Strengths:
- **Content Creation:** Jessica's experience in creating blog posts and social media updates could be a valuable asset, as these skills are essential for digital marketing roles.
- **Social Media Management:** Her success in growing social media follower numbers shows her capability in community engagement and content scheduling.
- **Market Research:** Experience in conducting market research and competitor analysis is advantageous and demonstrates analytical skills that can be applied to campaign planning.
- **Adaptability and Support Role:** Her ability to handle multiple tasks and provide general marketing support indicates a team-oriented approach and flexibility, which are positive traits in a fast-paced digital marketing environment.

### Recommendations for Improvement:
- Enhance the resume by including specific examples of campaigns managed, metrics achieved, and tools used, especially for Google Analytics and SEM.
- Consider taking additional courses or certifications related to Google Analytics and SEM to strengthen technical skills.
- Include any experience or knowledge related to CRM systems or Adobe Creative Suite to better align with job requirements.
- Expand on communication skills by highlighting specific writing projects or successful campaigns that showcase these abilities.

In [83]:
print_markdown("#### Gemini AI:")
print_markdown(gap_analysis_openai)

#### Gemini AI:

## Digital Marketing Specialist Resume Analysis: Jessica Brown

Here's an analysis of Jessica Brown's resume against the provided Digital Marketing Specialist job description:

**1. Key Requirements from Job Description:**

*   **Education:** Bachelor's degree in Marketing, Communications, or similar.
*   **Experience:** 2+ years of digital marketing experience.
*   **Skills:**
    *   SEO
    *   SEM
    *   Google Analytics
    *   Social Media Management
    *   Data Interpretation (basic marketing data)
    *   Communication and Writing Skills
    *   CRM Systems (e.g., HubSpot) - Helpful
    *   Adobe Creative Suite - Beneficial
*   **Responsibilities Alignment:** Experience in planning/executing campaigns, content creation, performance measurement, and coordinating with designers.

**2. Relevant Experience in Resume:**

*   **Education:** Bachelor of Commerce, Marketing - Directly aligns with the requirement.
*   **Experience:**
    *   2 years of experience as a Marketing Assistant - Meets the minimum experience requirement.
    *   Assisted with digital marketing campaigns including email and social media - Aligns with campaign execution responsibilities.
    *   Created blog posts and social media updates - Demonstrates content creation skills.
    *   Managed social media accounts and grew follower numbers - Shows social media management experience.
*   **Skills:**
    *   Digital Marketing (SEO basics, Email Marketing) - Partially aligns with SEO and Email Marketing requirements.
    *   Social Media Tools (Hootsuite, Buffer) - Demonstrates practical experience with social media management.

**3. Gaps/Mismatches:**

*   **SEM:** No mention of Search Engine Marketing (SEM) experience.
*   **Google Analytics:** No explicit mention of Google Analytics experience or preparing performance reports. While she "assisted with digital marketing campaigns," the resume doesn't highlight her role in measuring and reporting on campaign performance.
*   **Data Interpretation:** The resume doesn't showcase her ability to interpret marketing data.
*   **CRM Systems:** No mention of experience with CRM systems like HubSpot.
*   **Adobe Creative Suite:** Lists "Basic knowledge of Adobe Photoshop," but the job description mentions "Adobe Creative Suite" which includes other programs such as Illustrator and InDesign.
*   **Keyword Research:** No mention of experience with keyword research.
*   **Specificity:** While the resume mentions "assisting" with campaigns, it lacks quantifiable achievements or specific examples of her contributions and impact.

**4. Potential Strengths:**

*   **Social Media Growth:** The resume mentions growing follower numbers on social media. Quantifying this growth (e.g., "Increased Instagram followers by 30% in six months") would be a significant strength.
*   **Location:** Living in Toronto is a practical advantage, although not explicitly mentioned in the job description.
*   **Market Research and Competitor Analysis:** While not a core requirement, this experience demonstrates analytical skills and a broader understanding of the marketing landscape. She could potentially highlight how this research informed her digital marketing efforts.
*   **Transferable Skills:** Skills like Microsoft Office Suite and Google Workspace are valuable assets for general office productivity and collaboration, even though they are not the main focus of the role.


In [17]:
class ResumeOutput(BaseModel):
    updated_resume: str
    diff_html: str


def generate_resume(
    job_description_text: str, resume_text: str, gap_analysis_openai: str, model: str = "openai") -> dict:
    """
    Generate a tailored resume using OpenAI or Gemini.

    Args:
        job_description_text (str): The job description text.
        resume_text (str): The candidate's resume text.
        gap_analysis_openai (str): The gap analysis result from OpenAI.
        model (str): The model to use for resume generation.

    Returns:
        dict: A dictionary containing the updated resume and diff markdown.
    """
    # Construct the prompt for the AI model to generate the tailored resume.
    # The prompt includes context, instructions, and input data (original resume,
    # target job description, and gap analysis).
    prompt = (
        """
    ### Context:
    You are an expert resume writer and editor. Your goal is to rewrite the original resume to match the target job description, using the provided tailoring suggestions and analysis.

    ---

    ### Instruction:
    1. Rewrite the entire resume to best match the **Target Job Description** and **Gap Analysis to the Job Description**.
    2. Improve clarity, add job-relevant keywords, and quantify achievements.
    3. Specifically address the gaps identified in the analysis by:
       - Adding missing skills and technologies mentioned in the job description
       - Reframing experience to highlight relevant accomplishments
       - Strengthening sections that were identified as weak in the analysis
    4. Prioritize addressing the most critical gaps first
    5. Incorporate industry-specific terminology from the job description

     6. Ensure all quantifiable achievements are properly highlighted with metrics
    7. Return two versions of the resume:
        - `updated_resume`: The final rewritten resume (as plain text)
        - `diff_html`: A version of the resume with inline highlights using color:
            - Additions or rewritten content should be **green**:
            `<span style="color:green">your added or changed text</span>`
            - Removed content should be **red and struck through**:
            `<span style="color:red;text-decoration:line-through">removed text</span>`
            - Leave unchanged lines unmarked.
        - Keep all section headers and formatting consistent with the original resume.

    ---

    ### Output Format:

    ```json
    {
    "updated_resume": "<full rewritten resume as plain text>",
    "diff_markdown": "<HTML-colored version of the resume highlighting additions and deletions>"
    }
    ```
    ---
    ### Input:

    **Original Resume:**

    """
        + resume_text
        + """


    **Target Job Description:**

    """
        + job_description_text
        + """


    **Analysis of Resume vs. Job Description:**

    """
        + gap_analysis_openai
    )

    # Depending on the selected model, call the appropriate function to generate the resume.
    # If the OpenAI model is selected, it uses a temperature of 0.7 for creativity.
    if model == "openai":
        updated_resume_json = generate_resume(
        job_description_text,
        resume_text,
        gap_analysis_openai,
        model="openai"  # Change from "gemini" to "openai"
    )
    # If the Gemini model is selected, it uses a lower temperature of 0.5 for more focused results.
    elif model == "gemini":
        updated_resume_json = gemini_generate(prompt, temperature = 0.7)
    else:
        # Raise an error if an invalid model name is provided.
        raise ValueError(f"Invalid model: {model}")

    # Return the generated resume output as a dictionary.
    return updated_resume_json

In [109]:
updated_resume_json = generate_resume(job_description_text, resume_text, gap_analysis_openai, model="openai")
# Display the updated resume in Markdown format.
print_markdown(updated_resume_json.updated_resume)

**Jessica Brown**  
jessica.brown@email.com | (416) 555-7890 | linkedin.com/in/jessicabrown  

**Summary**  
Dynamic Digital Marketing Specialist with over 2 years of experience in developing and executing impactful digital marketing campaigns across multiple channels, including SEO, SEM, and social media. Proven ability to analyze performance data and optimize strategies to enhance audience engagement and drive brand growth. Strong communication skills and adept at collaborating with cross-functional teams to deliver creative solutions.  

**Experience**  

**Digital Marketing Assistant | Brewster Coffee Co. | Toronto, ON | Jan 2022 – Present**  
- Spearheaded digital marketing campaigns, including email and social media, resulting in a **30% increase in audience engagement** over six months.  
- Developed and executed SEO strategies, including keyword research and content optimization, which improved organic search rankings by **25%**.  
- Managed and optimized social media accounts, increasing follower count by **40%** across platforms in under a year.  
- Conducted market research and competitor analysis to inform campaign strategies and identify growth opportunities.  
- Utilized Google Analytics to track campaign performance, generating comprehensive reports to guide future marketing efforts.  
- Coordinated with designers to create visually appealing campaign materials, enhancing overall campaign effectiveness.  

**Skills**  
- Digital Marketing (SEO, SEM, Email Marketing)  
- Google Analytics, Data Interpretation  
- Social Media Management (Hootsuite, Buffer)  
- CRM Systems (HubSpot)  
- Adobe Creative Suite (Photoshop, Illustrator, InDesign)  
- Microsoft Office Suite, Google Workspace  

**Education**  
**Bachelor of Commerce, Marketing** | Ryerson University (now Toronto Metropolitan University), Toronto, ON | May 2021

In [117]:
print_markdown(updated_resume_json.diff_html)  # Change from diff_markdown to diff_html

AttributeError: 'ResumeOutput' object has no attribute 'diff_html'

In [118]:
print(dir(updated_resume_json))  # This will show you what attributes exist

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_computed_fields__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic_serializer__', '__pydantic_setattr_handlers__', '__pydantic_validator__', '__reduce__', 

In [119]:
# Option 1: If it still has diff_markdown
print_markdown(updated_resume_json.diff_markdown)

# Option 2: If you updated the model but need to re-run the generation
updated_resume_json = generate_resume(job_description_text, resume_text, gap_analysis_openai, model="openai")
print_markdown(updated_resume_json.diff_html)

**Jessica Brown**  
jessica.brown@email.com | (416) 555-7890 | linkedin.com/in/jessicabrown  

**Summary**  
<span style="color:green">Dynamic Digital Marketing Specialist with over 2 years of experience in developing and executing impactful digital marketing campaigns across multiple channels, including SEO, SEM, and social media. Proven ability to analyze performance data and optimize strategies to enhance audience engagement and drive brand growth. Strong communication skills and adept at collaborating with cross-functional teams to deliver creative solutions.</span>  

**Experience**  

**Digital Marketing Assistant | Brewster Coffee Co. | Toronto, ON | Jan 2022 – Present**  
- <span style="color:green">Spearheaded digital marketing campaigns, including email and social media, resulting in a 30% increase in audience engagement over six months.</span>  
- <span style="color:green">Developed and executed SEO strategies, including keyword research and content optimization, which improved organic search rankings by 25%.</span>  
- <span style="color:green">Managed and optimized social media accounts, increasing follower count by 40% across platforms in under a year.</span>  
- Conducted market research and competitor analysis to inform campaign strategies and identify growth opportunities.  
- <span style="color:green">Utilized Google Analytics to track campaign performance, generating comprehensive reports to guide future marketing efforts.</span>  
- <span style="color:green">Coordinated with designers to create visually appealing campaign materials, enhancing overall campaign effectiveness.</span>  

**Skills**  
- Digital Marketing (SEO, SEM, Email Marketing)  
- <span style="color:green">Google Analytics, Data Interpretation</span>  
- Social Media Management (Hootsuite, Buffer)  
- <span style="color:green">CRM Systems (HubSpot)</span>  
- <span style="color:green">Adobe Creative Suite (Photoshop, Illustrator, InDesign)</span>  
- Microsoft Office Suite, Google Workspace  

**Education**  
**Bachelor of Commerce, Marketing** | Ryerson University (now Toronto Metropolitan University), Toronto, ON | May 2021

RecursionError: maximum recursion depth exceeded

In [18]:
class CoverLetterOutput(BaseModel):
    cover_letter: str

# The generate_cover_letter function creates a cover letter based on the provided job description and updated resume.
# It takes three parameters:
# (1) job_description_text: A string containing the job description for the position.
# (2) updated_resume: A string containing the candidate's updated resume.
# (3) model: A string indicating which model to use for generating the cover letter (default is "openai").
# The function returns a dictionary containing the generated cover letter.

def generate_cover_letter(job_description_text: str, updated_resume: str, model: str = "openai") -> dict:
    """
    Generate a cover letter using OpenAI or Gemini.

    Args:
        job_description_text (str): The job description text.
        updated_resume (str): The candidate's updated resume text.
        model (str): The model to use for cover letter generation.

    Returns:
        dict: A dictionary containing the cover letter.
    """

    # Construct the prompt for the AI model, including context and instructions for writing the cover letter.
    prompt = (
        """
    ### Context:
    You are a professional career coach and expert cover letter writer.

    ---

    ### Instruction:
    Write a compelling, personalized cover letter based on the **Updated Resume** and the **Target Job Description**. The letter should:
    1. Be addressed generically (e.g., "Dear Hiring Manager")
    2. Be no longer than 4 paragraphs
    3. Highlight key achievements and experiences from the updated resume
    4. Align with the responsibilities and qualifications in the job description
    5. Reflect the applicant's enthusiasm and fit for the role
    6. End with a confident and polite closing statement

    ---
    ### Output Format (JSON):
    ```json
    {
    "cover_letter": "<final cover letter text>"
    }
    ```
    ---

    ### Input:

    **Updated Resume:**

    """
        + updated_resume
        + """
    **Target Job Description:**

    """
        + job_description_text
    )

    # Depending on the selected model, call the appropriate function to generate the cover letter.
    if model == "openai":
        # Get response from OpenAI API
        updated_cover_letter = openai(prompt, temperature=0.7, response_format=CoverLetterOutput)
    elif model == "gemini":
        # Get response from Gemini API
        updated_cover_letter = gemini_generate(prompt, temperature=0.5)
    else:
        # Raise an error if an invalid model name is provided.
        raise ValueError(f"Invalid model: {model}")

    # Return the generated cover letter as a dictionary.
    return updated_cover_letter


In [123]:
updated_cover_letter = generate_cover_letter(job_description_text, updated_resume_json.updated_resume, model="openai")

# Display the generated cover letter in Markdown format.
print_markdown(updated_cover_letter.cover_letter)

Dear Hiring Manager,

I am excited to apply for the Digital Marketing Specialist position at BrightWave Digital Agency. With a Bachelor’s degree in Marketing and over two years of experience as a Digital Marketing Assistant at Brewster Coffee Co., I have honed my skills in developing and executing effective digital marketing campaigns across various platforms, making me a strong candidate for this role.

During my time at Brewster Coffee Co., I successfully spearheaded multiple campaigns that resulted in a 30% increase in audience engagement within just six months. My experience in optimizing SEO strategies contributed to a 25% improvement in organic search rankings, aligning perfectly with your need for someone to assist in planning and executing digital marketing campaigns. Additionally, I am proficient in using Google Analytics to track performance metrics and prepare insightful reports, which is essential for the responsibilities outlined in your job description.

I have a proven track record in social media management, having increased our follower count by 40% across various platforms while also engaging with the community to drive brand loyalty. My collaborative skills enable me to work effectively with designers to create visually appealing materials that resonate with target audiences. I am also passionate about staying updated with the latest digital marketing trends, ensuring that I bring fresh ideas to the team.

I am eager to contribute my skills in digital marketing and my passion for creative solutions to BrightWave Digital Agency. I look forward to the opportunity to discuss how my background and enthusiasms align with the goals of your team. Thank you for considering my application.

Sincerely,

Jessica Brown

In [19]:
def run_resume_rocket(resume_text: str, job_description_text: str) -> tuple[str, str]:
    """
    Run the resume rocket workflow.

    Args:
        resume_text (str): The candidate's resume text.
        job_description_text (str): The job description text.

    Returns:
        tuple: A tuple containing the updated resume and cover letter.
    """
    # Analyze the candidate's resume against the job description using OpenAI's model.
    # This function will return a structured analysis highlighting gaps and strengths.
    gap_analysis_openai = analyze_resume_against_job_description(job_description_text,
                                                                 resume_text,
                                                                 model="openai")

    # Display the gap analysis results in Markdown format for better readability.
    print_markdown(gap_analysis_openai)

    # Print separators for clarity in the output.
    print("\n--------------------------------")
    print("--------------------------------\n")

    # Generate an updated resume based on the job description, original resume, and gap analysis.
    # This function will return a JSON-like object containing the updated resume and a diff markdown.
    updated_resume_json = openai(prompt, response_format=ResumeOutput, temperature=0.7)

    # Display the diff markdown which shows the changes made to the resume.
    # print_markdown(updated_resume_json.diff_markdown)
    print("\n--------------------------------")
    print("--------------------------------\n")

    # Display the updated resume in Markdown format.
    print_markdown(updated_resume_json.updated_resume)

    # Print separators for clarity in the output.
    print("\n--------------------------------")
    print("--------------------------------\n")

    # Generate a cover letter based on the job description and the updated resume.
    # This function will return the generated cover letter.
    updated_cover_letter = generate_cover_letter(
        job_description_text, updated_resume_json.updated_resume, model="openai"
    )

    # Display the generated cover letter in Markdown format.
    print_markdown(updated_cover_letter.cover_letter)

    # Print separators for clarity in the output.
    print("\n--------------------------------")
    print("--------------------------------\n")

    # Return the updated resume and the generated cover letter as a tuple.
    return updated_resume_json.updated_resume, updated_cover_letter.cover_letter



In [20]:
resume, cover_letter = run_resume_rocket(resume_text, job_description_text)

### 1. Key Requirements from Job Description:
- **Education**: Bachelor's degree in Marketing, Communications, or similar.
- **Experience**: 2+ years of digital marketing experience.
- **Skills**:
  - Planning and executing digital marketing campaigns (SEO, SEM, social media, email).
  - Familiarity with Google Analytics and ability to prepare performance reports.
  - Social media management, content scheduling, and community engagement.
  - Keyword research and SEO content optimization.
  - Collaboration with designers on campaign materials.
  - Awareness of current digital marketing trends.
- **Communication**: Strong communication and writing skills.
- **Technical Skills**: Knowledge of CRM systems (e.g., HubSpot) and familiarity with Adobe Creative Suite.

### 2. Relevant Experience in Resume:
- **Education**: Bachelor of Commerce in Marketing.
- **Experience**: 2 years as a Marketing Assistant, assisting in digital marketing campaigns, particularly email and social media.
- **Skills**:
  - Experience with social media management and content creation.
  - Basic understanding of SEO and email marketing.
  - Conducted market research and competitor analysis, indicating an understanding of digital marketing trends.
- **Technical Skills**: 
  - Familiarity with social media tools (Hootsuite, Buffer).
  - Basic knowledge of Adobe Photoshop.

### 3. Gaps/Mismatches:
- **Experience Duration**: The candidate has only 2 years of experience, which is on the edge of the required experience but not exceeding it; this could be a concern.
- **Specific Tools and Skills**:
  - The candidate lacks direct mention of Google Analytics experience, which is specifically requested.
  - No experience with SEM (Search Engine Marketing) is mentioned.
  - No mention of CRM systems (e.g., HubSpot), which is considered helpful.
  - The candidate's experience with Adobe Creative Suite is not strong as it only mentions Adobe Photoshop, while broader knowledge is preferred.
- **Performance Reporting**: There is no indication of experience preparing performance reports, which is an important aspect of the role.

### 4. Potential Strengths:
- **Social Media Management**: The candidate shows a proactive approach in managing social media accounts and improving audience engagement, which is valuable for a digital marketing role.
- **Content Creation**: Experience in creating blog posts and social media updates demonstrates writing skills and creativity, which could be a strong asset.
- **Market Research**: Conducting market research and competitor analysis indicates analytical skills and an understanding of the marketing landscape, which is beneficial.
- **Adaptability**: The ability to handle multiple tasks and provide general marketing support suggests good organizational and multitasking skills.

### Summary:
Jessica Brown's resume presents a solid foundation for the Digital Marketing Specialist position at BrightWave Digital Agency. While she meets the educational requirement and has relevant experience in digital marketing, there are notable gaps in specific skills, particularly with Google Analytics, SEM, and CRM systems. Strengths in social media management and content creation can enhance her application, but addressing the gaps and emphasizing transferable skills will be essential for a stronger alignment with the job description.


--------------------------------
--------------------------------


--------------------------------
--------------------------------



**Jessica Brown**  
jessica.brown@email.com | (416) 555-7890 | linkedin.com/in/jessicabrown  

---  

**Summary**  
Results-driven marketing professional with over 2 years of experience in executing impactful digital campaigns, creating engaging content, and elevating brand presence on social media. Demonstrated ability to manage multiple projects simultaneously while delivering measurable results and enhancing audience engagement.  

---  

**Experience**  

**Marketing Assistant | Brewster Coffee Co. | Toronto, ON | Jan 2022 – Present**  
- Spearheaded digital marketing campaigns that resulted in a 30% increase in email open rates and a 25% rise in social media engagement.  
- Developed and published 20+ blog posts and social media updates monthly, boosting website traffic by 40% year-over-year.  
- Managed and optimized social media accounts, growing follower base from 1,000 to 3,500 in 12 months through targeted content strategies.  
- Coordinated and executed 5 successful marketing events, enhancing brand visibility and generating a 15% increase in customer inquiries.  
- Conducted comprehensive market research and competitor analysis, informing strategic decisions and improving campaign effectiveness.  

---  

**Skills**  
- **Digital Marketing:** Proficient in SEO, Email Marketing, and content strategy development.  
- **Social Media Management:** Experienced with Hootsuite and Buffer for scheduling and analytics.  
- **Technical Skills:** Microsoft Office Suite, Google Workspace, and foundational Adobe Photoshop.  
- **Analytical Skills:** Strong ability to interpret data and metrics to drive campaign improvements.  

---  

**Education**  
**Bachelor of Commerce, Marketing**  
Ryerson University (now Toronto Metropolitan University), Toronto, ON | May 2021  

---


--------------------------------
--------------------------------



Dear Hiring Manager,

I am writing to express my enthusiasm for the Digital Marketing Specialist position at BrightWave Digital Agency, as advertised. With a Bachelor’s degree in Marketing and over two years of hands-on experience in digital marketing, I am excited about the opportunity to contribute to your team by driving impactful campaigns and enhancing brand visibility for your diverse clientele.

At Brewster Coffee Co., I successfully spearheaded digital marketing campaigns that led to a 30% increase in email open rates and a 25% rise in social media engagement. My role involved developing and publishing over 20 blog posts and social media updates each month, which boosted website traffic by 40% year-over-year. These experiences have equipped me with a solid foundation in SEO, SEM, and content strategy development, aligning perfectly with the responsibilities outlined in your job description.

Additionally, my proficiency in Google Analytics has enabled me to interpret data effectively and prepare performance reports that inform strategic decisions. I have also managed social media accounts, growing our follower base significantly through targeted content strategies and community engagement. I am keen to bring this expertise to BrightWave, supporting your clients in achieving their marketing goals.

I am particularly drawn to BrightWave's commitment to innovation in digital marketing, and I am eager to contribute to your continued success while continuing to grow in my career. Thank you for considering my application. I look forward to the opportunity to discuss how my background and skills align with the needs of your team.

Sincerely,
Jessica Brown


--------------------------------
--------------------------------



In [22]:
new_resume_text = """
JOHN SMITH
Data Scientist | john.smith@email.com | (555) 123-4567 | linkedin.com/in/johnsmith

SUMMARY
Experienced data scientist with 5 years of experience in machine learning, statistical analysis, and data visualization. Proficient in Python, SQL, and R with a track record of delivering actionable insights.

SKILLS
- Programming: Python, R, SQL
- Machine Learning: Scikit-learn, TensorFlow, PyTorch
- Data Visualization: Tableau, Matplotlib, Seaborn
- Tools: Git, Jupyter, Docker

EXPERIENCE
Senior Data Scientist | ABC Analytics | 2020-Present
- Led a team of 3 data scientists in developing predictive models that increased customer retention by 15%
- Implemented NLP solutions for automated text classification with 92% accuracy
- Optimized data pipelines reducing processing time by 30%

Data Analyst | XYZ Corporation | 2018-2020
- Conducted A/B testing to optimize website conversion rates, resulting in 25% increase
- Created interactive dashboards for executive team using Tableau
- Automated reporting processes saving 10 hours per week

EDUCATION
Master of Science in Data Science | University of Data | 2018
Bachelor of Science in Computer Science | Tech University | 2016
"""

new_job_description_text = """
SENIOR DATA ENGINEER

Company: Tech Innovations Inc.
Location: Remote (US-based)

Job Description:
We are seeking an experienced Senior Data Engineer to design, build and maintain our data infrastructure. The ideal candidate will have strong experience with cloud platforms, data warehousing, and ETL processes.

Responsibilities:
- Design and implement scalable data pipelines and ETL processes
- Build and optimize data models for analytics and reporting
- Collaborate with data scientists to productionize machine learning models
- Maintain data quality and integrity across all systems
- Implement data security and compliance measures

Requirements:
- 5+ years of experience in data engineering
- Expert knowledge of SQL and Python
- Experience with AWS services (S3, Redshift, Glue, Lambda)
- Familiarity with data warehousing concepts and dimensional modeling
- Experience with Apache Spark, Airflow, or similar technologies
- Strong problem-solving skills and attention to detail
- Excellent communication skills

Preferred Qualifications:
- Experience with real-time data processing
- Knowledge of machine learning deployment
- Familiarity with data governance frameworks
- Experience with Kubernetes and Docker
"""

In [23]:
new_resume_text,new_cover_letter = run_resume_rocket(new_resume_text, new_job_description_text)

### 1. Key Requirements from Job Description:

- **Experience:**
  - 5+ years of experience in data engineering
- **Technical Skills:**
  - Expert knowledge of SQL and Python
  - Experience with AWS services (S3, Redshift, Glue, Lambda)
  - Familiarity with data warehousing concepts and dimensional modeling
  - Experience with Apache Spark, Airflow, or similar technologies
- **Responsibilities:**
  - Design and implement scalable data pipelines and ETL processes
  - Build and optimize data models for analytics and reporting
  - Collaborate with data scientists to productionize machine learning models
  - Maintain data quality and integrity across all systems
  - Implement data security and compliance measures
- **Soft Skills:**
  - Strong problem-solving skills and attention to detail
  - Excellent communication skills
- **Preferred Qualifications:**
  - Experience with real-time data processing
  - Knowledge of machine learning deployment
  - Familiarity with data governance frameworks
  - Experience with Kubernetes and Docker

### 2. Relevant Experience in Resume:

- **Experience:**
  - 5 years of experience in data science, which may overlap with data engineering tasks.
- **Technical Skills:**
  - Proficient in Python and SQL, as required.
  - Experience with Docker, which is mentioned in the job description.
- **Responsibilities:**
  - Optimized data pipelines in the current role, which aligns with designing and implementing scalable data pipelines.
  - Created interactive dashboards and automated reporting processes, indicating experience with data modeling for analytics.
- **Soft Skills:**
  - Leadership experience as a team lead, indicating strong communication and problem-solving skills.

### 3. Gaps/Mismatches:

- **Experience in Data Engineering:** The resume primarily focuses on data science rather than specific data engineering experience, which may not fully meet the "5+ years of experience in data engineering" requirement.
- **AWS Services:** No mention of experience with AWS services such as S3, Redshift, Glue, or Lambda, which is critical for the role.
- **Data Warehousing Concepts:** Lack of specific mention of familiarity with data warehousing concepts and dimensional modeling.
- **Experience with ETL Processes:** While there is some mention of optimizing data pipelines, there is no explicit experience detailed with ETL processes.
- **Technologies:** No mention of experience with Apache Spark, Airflow, or similar technologies.
- **Real-time Data Processing:** There is no indication of experience with real-time data processing.
- **Data Security and Compliance:** No mention of experience implementing data security and compliance measures.

### 4. Potential Strengths:

- **Machine Learning Expertise:** The candidate has a solid background in machine learning, which could be valuable in collaborating with data scientists to productionize models.
- **Leadership Experience:** Leading a team of data scientists demonstrates leadership and project management abilities, which may be beneficial in a senior role.
- **Statistics and Data Visualization Skills:** Strong skills in statistical analysis and data visualization tools can enhance data interpretation and reporting, adding value beyond basic data engineering tasks.
- **Automated Reporting Processes:** Experience in automation can contribute to efficiency and optimization in data processes.

### Summary:

While the candidate has relevant skills and experience in data science, there are significant gaps in data engineering experience, particularly concerning AWS services, ETL processes, and data warehousing concepts. Emphasizing relevant skills and seeking additional experience or certifications in data engineering, particularly with cloud services, could strengthen the application for this Senior Data Engineer position.


--------------------------------
--------------------------------


--------------------------------
--------------------------------



**Jessica Brown**  
[jessica.brown@email.com](mailto:jessica.brown@email.com) | (416) 555-7890 | [linkedin.com/in/jessicabrown](https://linkedin.com/in/jessicabrown)  

---  
**Summary**  
Dynamic marketing professional with over 2 years of experience in driving successful digital campaigns, creating compelling content, and enhancing brand visibility through social media strategies. Proven ability to manage multiple projects simultaneously while delivering measurable results in audience engagement and brand growth.  

---  
**Experience**  

**Marketing Assistant | Brewster Coffee Co. | Toronto, ON | Jan 2022 – Present**  
- Spearheaded digital marketing campaigns, achieving a **30% increase** in email open rates through targeted content and segmentation strategies.  
- Developed and executed engaging blog posts and social media updates, resulting in a **50% boost** in audience engagement across platforms.  
- Managed and optimized social media accounts, successfully growing the follower base by **40%** in under a year.  
- Coordinated marketing events, enhancing brand presence and driving a **20% increase** in foot traffic during promotional campaigns.  
- Conducted in-depth market research and competitor analysis, providing actionable insights that informed strategic marketing decisions.  

---  
**Skills**  
- **Digital Marketing:** SEO fundamentals, Email Marketing strategies  
- **Social Media Management:** Proficient in Hootsuite, Buffer, and analytics tools  
- **Technical Skills:** Microsoft Office Suite, Google Workspace, Basic Adobe Photoshop  
- **Research & Analysis:** Market research, Competitive analysis  

---  
**Education**  
**Bachelor of Commerce in Marketing**  
Ryerson University (now Toronto Metropolitan University), Toronto, ON | May 2021  

---


--------------------------------
--------------------------------



Dear Hiring Manager,

I am writing to express my interest in the Senior Data Engineer position at Tech Innovations Inc. While I have a background in marketing and digital strategy, I am eager to pivot my career into data engineering where I can leverage my analytical skills and passion for data management. During my time as a Marketing Assistant at Brewster Coffee Co., I gained valuable experience in data analysis and project management, which I believe will serve as a solid foundation for the responsibilities outlined in your job description.

In my current role, I spearheaded digital marketing campaigns that required me to analyze engagement data and optimize strategies based on these insights. This experience honed my ability to manage and interpret large datasets, resulting in a 30% increase in email open rates and a 50% boost in audience engagement. My proactive approach to market research and competitor analysis has also provided me with a keen understanding of data quality and integrity, which I know is crucial for maintaining robust data systems.

Although my professional background is not strictly in data engineering, I possess a strong aptitude for learning new technologies and methodologies. I am proficient in SQL and have a foundational understanding of Python, which I have utilized in various projects to enhance data-driven decision-making. My enthusiasm for continuous learning drives me to explore cloud platforms and data warehousing concepts, positioning me to quickly adapt and excel in the technical aspects of the Senior Data Engineer role.

I am excited about the opportunity to contribute to the innovative projects at Tech Innovations Inc. and am confident that my unique perspective and dedication to data integrity will be an asset to your team. Thank you for considering my application. I look forward to the possibility of discussing how I can help drive successful data initiatives within your organization.

Sincerely,
Jessica Brown
[jessica.brown@email.com](mailto:jessica.brown@email.com)  
(416) 555-7890


--------------------------------
--------------------------------

